<a href="https://colab.research.google.com/github/rajaranjith/HCL-GenAI-Training/blob/main/Assignment_1_Gold_Badge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Install dependencies
# Install the official Mistral Python SDK
!pip install -q mistralai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.0/461.0 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 3.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.21.0 requires opentelemetry-api<=1.37.0,>=1.37.0, but you have opentelemetry-api 1.38.0 which is incompatible.
google-adk 1.21.0 requires opentelemetry-sdk<=1.37.0,>=1.37.0, but you have opentelemetry-sdk 1.38.0 which is incompatible.


In [2]:
#Set up API key and client
import os
import getpass
from mistralai import Mistral

# Ask for your API key once per Colab session
if "MISTRAL_API_KEY" not in os.environ or not os.environ["MISTRAL_API_KEY"]:
    os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter your Mistral API key: ")

api_key = os.environ["MISTRAL_API_KEY"]
client = Mistral(api_key=api_key)

print("✅ Mistral client initialized.")

Enter your Mistral API key: ··········
✅ Mistral client initialized.


In [15]:
#Create a tiny sentiment dataset and save as JSONL
import random

all_examples = [
    # (text, label) pairs
    ("I love this phone, the battery lasts all day.", "positive"),
    ("Terrible experience. The screen broke in two days.", "negative"),
    ("Amazing camera and very fast performance!", "positive"),
    ("The app keeps crashing, I'm very disappointed.", "negative"),
    ("Works fine but the sound quality could be better.", "negative"),
    ("Best purchase I've made this year.", "positive"),
    ("Really happy with this laptop overall.", "positive"),
    ("Customer support was rude and unhelpful.", "negative"),
    ("This headset is comfortable and sounds great.", "positive"),
    ("The product stopped working after a week.", "negative"),
    ("Pretty good overall, just a bit heavy.", "positive"),
    ("Worst customer service I've ever seen.", "negative"),
    ("Battery life is awful and it overheats.", "negative"),
    ("I’m impressed, it works better than expected.", "positive"),
    ("It's okay, but not very durable.", "negative"),
    ("Excellent quality, will recommend to friends.", "positive"),
    ("Terrible packaging, item arrived broken.", "negative"),
    ("I'm satisfied with the purchase.", "positive"),
    ("Too slow and the interface is confusing.", "negative"),
    ("The build quality feels premium.", "positive"),
]

# Shuffle so train/val are not ordered
random.shuffle(all_examples)

split_idx = int(0.8 * len(all_examples))  # 80% train, 20% val
train_examples = all_examples[:split_idx]
eval_examples  = all_examples[split_idx:]

print("Train examples:", len(train_examples))
print("Eval examples :", len(eval_examples))

INSTRUCTION = (
    "You are a sentiment classifier.\n"
    "Reply with EXACTLY one word: 'positive' or 'negative'.\n"
    "No explanations, no punctuation, just the label.\n\n"
    "Text: "
)

def write_jsonl(filename, pairs):
    import json
    with open(filename, "w", encoding="utf-8") as f:
        for text, label in pairs:
            record = {
                "messages": [
                    {"role": "user", "content": INSTRUCTION + text},
                    {"role": "assistant", "content": label},
                ]
            }
            f.write(json.dumps(record, ensure_ascii=False) + "\n")

write_jsonl("sentiment_train.jsonl", train_examples)
write_jsonl("sentiment_eval.jsonl", eval_examples)

print("✅ Wrote sentiment_train.jsonl and sentiment_eval.jsonl")

# Quick sanity check – show first 2 lines
!head -n 2 sentiment_train.jsonl

Train examples: 16
Eval examples : 4
✅ Wrote sentiment_train.jsonl and sentiment_eval.jsonl
{"messages": [{"role": "user", "content": "You are a sentiment classifier.\nReply with EXACTLY one word: 'positive' or 'negative'.\nNo explanations, no punctuation, just the label.\n\nText: I'm satisfied with the purchase."}, {"role": "assistant", "content": "positive"}]}
{"messages": [{"role": "user", "content": "You are a sentiment classifier.\nReply with EXACTLY one word: 'positive' or 'negative'.\nNo explanations, no punctuation, just the label.\n\nText: Really happy with this laptop overall."}, {"role": "assistant", "content": "positive"}]}


In [16]:
#Upload dataset files to Mistral
# Upload the training and validation files to Mistral Cloud
train_file = client.files.upload(
    file={
        "file_name": "sentiment_train.jsonl",
        "content": open("sentiment_train.jsonl", "rb"),
    }
)

eval_file = client.files.upload(
    file={
        "file_name": "sentiment_eval.jsonl",
        "content": open("sentiment_eval.jsonl", "rb"),
    }
)

print("✅ Uploaded files.")
print("Training file id:", train_file.id)
print("Eval file id    :", eval_file.id)

✅ Uploaded files.
Training file id: 9375d388-6af7-4b89-a679-ec448db23947
Eval file id    : b6b9abf8-ee48-412d-ba6f-70610c17a069


In [17]:
#(Optional) Pretty-print file metadata
import json

def pprint_obj(o):
    print(json.dumps(o.model_dump(), indent=2))

print("Training file:")
pprint_obj(train_file)

print("\nValidation file:")
pprint_obj(eval_file)

Training file:
{
  "id": "9375d388-6af7-4b89-a679-ec448db23947",
  "object": "file",
  "bytes": null,
  "created_at": 1766818024,
  "filename": "sentiment_train.jsonl",
  "purpose": "fine-tune",
  "sample_type": "instruct",
  "source": "upload",
  "num_lines": 16,
  "mimetype": "application/jsonl",
  "signature": "0bb56898700a4e8c"
}

Validation file:
{
  "id": "b6b9abf8-ee48-412d-ba6f-70610c17a069",
  "object": "file",
  "bytes": null,
  "created_at": 1766818024,
  "filename": "sentiment_eval.jsonl",
  "purpose": "fine-tune",
  "sample_type": "instruct",
  "source": "upload",
  "num_lines": 4,
  "mimetype": "application/jsonl",
  "signature": "79981caa910567f1"
}


In [21]:
hyperparams = {
    "training_steps": 10,   # small for demo
    "learning_rate": 1e-4,
}

job = client.fine_tuning.jobs.create(
    model="open-mistral-7b",
    training_files=[{"file_id": train_file.id, "weight": 1}],
    validation_files=[eval_file.id],
    hyperparameters=hyperparams,
    auto_start=True,  # start as soon as it validates
)

print("✅ Created fine-tuning job.")
print("Job id:", job.id)

# Optional: inspect full job object
import json
print(json.dumps(job.model_dump(), indent=2))

✅ Created fine-tuning job.
Job id: c11ce996-729a-4608-80b8-2ab206d72d58
{
  "id": "c11ce996-729a-4608-80b8-2ab206d72d58",
  "auto_start": true,
  "model": "open-mistral-7b",
  "status": "QUEUED",
  "created_at": 1766818104,
  "modified_at": 1766818104,
  "training_files": [
    "9375d388-6af7-4b89-a679-ec448db23947"
  ],
  "hyperparameters": {
    "training_steps": 10,
    "learning_rate": 0.0001,
    "weight_decay": 0.1,
    "warmup_fraction": 0.05,
    "epochs": null,
    "seq_len": 32768,
    "fim_ratio": null
  },
  "validation_files": [
    "b6b9abf8-ee48-412d-ba6f-70610c17a069"
  ],
  "object": "job",
  "fine_tuned_model": null,
  "suffix": null,
  "integrations": [],
  "trained_tokens": null,
  "metadata": {
    "expected_duration_seconds": null,
    "cost": 0.0,
    "cost_currency": null,
    "train_tokens_per_step": null,
    "train_tokens": null,
    "data_tokens": null,
    "estimated_start_time": null
  },
  "job_type": "completion",
  "repositories": []
}


In [22]:
#Poll job status until it finishes
import time
import json

def pprint_obj(o):
    print(json.dumps(o.model_dump(), indent=2))

job_id = job.id

retrieved_job = client.fine_tuning.jobs.get(job_id=job_id)
print("Initial status:", retrieved_job.status)

while retrieved_job.status in ("QUEUED", "VALIDATED", "RUNNING"):
    pprint_obj(retrieved_job)
    print(f"Status is {retrieved_job.status} – checking again in 10 seconds...\n")
    time.sleep(10)
    retrieved_job = client.fine_tuning.jobs.get(job_id=job_id)

import json
print(json.dumps(retrieved_job.dict(), indent=2))

print("🚩 Final status:", retrieved_job.status)
if retrieved_job.status == "SUCCESS":
    print("🎉 Fine-tuned model:", retrieved_job.fine_tuned_model)
else:
    print("Job did not succeed – check job details above.")

Initial status: RUNNING
{
  "id": "c11ce996-729a-4608-80b8-2ab206d72d58",
  "auto_start": true,
  "model": "open-mistral-7b",
  "status": "RUNNING",
  "created_at": 1766818104,
  "modified_at": 1766818105,
  "training_files": [
    "9375d388-6af7-4b89-a679-ec448db23947"
  ],
  "hyperparameters": {
    "training_steps": 10,
    "learning_rate": 0.0001,
    "weight_decay": 0.1,
    "warmup_fraction": 0.05,
    "epochs": 1479.3679458239278,
    "seq_len": 32768,
    "fim_ratio": null
  },
  "validation_files": [
    "b6b9abf8-ee48-412d-ba6f-70610c17a069"
  ],
  "object": "job",
  "fine_tuned_model": null,
  "suffix": null,
  "integrations": [],
  "trained_tokens": null,
  "metadata": {
    "expected_duration_seconds": 450,
    "cost": 0.0,
    "cost_currency": null,
    "train_tokens_per_step": 131072,
    "train_tokens": 1310720,
    "data_tokens": 886,
    "estimated_start_time": null
  },
  "job_type": "completion",
  "repositories": [],
  "events": [
    {
      "name": "status-update

/tmp/ipython-input-3725116268.py:20: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  print(json.dumps(retrieved_job.dict(), indent=2))


In [26]:
#Use the fine-tuned model for inference
# Make sure the job actually succeeded
if retrieved_job.status != "SUCCESS":
    raise RuntimeError(f"Job status is {retrieved_job.status}, cannot run inference.")

fine_tuned_model = retrieved_job.fine_tuned_model
print("Using model:", fine_tuned_model)

test_texts = [
    "This is the best thing I've ever bought.",
    "Horrible quality, I'm never buying from them again.",
    "Pretty decent for the price.",
    "I hate how slow and buggy this is.",
]

for text in test_texts:
    resp = client.chat.complete(
        model=fine_tuned_model,
        messages=[
            {
                "role": "user",
                "content": INSTRUCTION + text,
            }
        ],
        # optional: make responses deterministic
        temperature=0.0,
    )

    # Safely extract text from the response
    resp_data = resp.model_dump() # Changed from resp.dict() to resp.model_dump()
    # The content is directly available as a string in the 'content' field
    prediction = resp_data["choices"][0]["message"]["content"].strip()

    print(f"\nText: {text}")
    print("Predicted label:", prediction)

Using model: ft:open-mistral-7b:bee045e5:20251227:c11ce996

Text: This is the best thing I've ever bought.
Predicted label: positive

Text: Horrible quality, I'm never buying from them again.
Predicted label: negative

Text: Pretty decent for the price.
Predicted label: positive

Text: I hate how slow and buggy this is.
Predicted label: negative
